Nama : Bernadus Sergio Halim
Nim : 0706022210056
Tanggal pemberian tugas : 7/3/2025


In [ ]:
%pip install ortools
!pip install --upgrade --no-cache-dir ortools


Routing Problems

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    data = {}
    data["distance_matrix"] = [
        [0, 10, 15, 20, 25, 30, 35],
        [10, 0, 35, 25, 30, 45, 50],
        [15, 35, 0, 30, 20, 25, 30],
        [20, 25, 30, 0, 15, 20, 25],
        [25, 30, 20, 15, 0, 35, 40],
        [30, 45, 25, 20, 35, 0, 15],
        [35, 50, 30, 25, 40, 15, 0],
    ]
    data["num_vehicles"] = 3
    data["depot"] = 0
    return data

def print_solution(manager, routing, solution):
    print("Objective: {}".format(solution.ObjectiveValue()))
    for vehicle_id in range(3):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
        print(f"Route for vehicle {vehicle_id}: {route}")

def main():
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(manager, routing, solution)

main()

Objective: 145
Route for vehicle 0: [0, 0]
Route for vehicle 1: [0, 0]
Route for vehicle 2: [0, 1, 3, 4, 2, 5, 6, 0]


Linear programing (LP)  

In [ ]:
from scipy.optimize import linprog

# Koefisien fungsi objektif (maximization -> minimization dengan negatif)
c = [-5000, -4000]  # Negatif karena linprog hanya bisa minimisasi

# Koefisien kendala
A = [
    [2, 4],   # Jam kerja
    [3, 2]    # Bahan baku
]

# Batasan kendala
b = [40, 30]

# Batasan variabel (x, y >= 0)
x_bounds = (0, None)
y_bounds = (0, None)

# Penyelesaian Linear Programming
result = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method='highs')

# Menampilkan hasil
if result.success:
    x_opt, y_opt = result.x
    max_profit = -result.fun  # Karena kita mengubah ke minimisasi
    print(f"Optimal Solution: Produk A = {x_opt:.2f}, Produk B = {y_opt:.2f}")
    print(f"Maksimum Keuntungan: Rp{max_profit:.2f}")
else:
    print("Solusi tidak ditemukan")



Optimal Solution: Produk A = 5.00, Produk B = 7.50
Maksimum Keuntungan: Rp55000.00


Constraint Programming (CP)

In [ ]:
from ortools.sat.python import cp_model

def solve_shift_scheduling():
    model = cp_model.CpModel()

    # Variabel untuk shift masing-masing karyawan
    A = model.NewIntVar(1, 3, 'A')  # Karyawan A
    B = model.NewIntVar(1, 3, 'B')  # Karyawan B
    C = model.NewIntVar(1, 3, 'C')  # Karyawan C

    # Kendala: Setiap karyawan harus memiliki shift yang berbeda
    model.AddAllDifferent([A, B, C])

    # Kendala: Karyawan A tidak boleh di shift malam (3)
    model.Add(A != 3)

    # Kendala: Karyawan B tidak boleh di shift pagi (1)
    model.Add(B != 1)

    # Solver
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        print(f"Karyawan A -> Shift {solver.Value(A)}")
        print(f"Karyawan B -> Shift {solver.Value(B)}")
        print(f"Karyawan C -> Shift {solver.Value(C)}")
    else:
        print("Tidak ada solusi yang memenuhi kendala.")

# Jalankan program
solve_shift_scheduling()


Karyawan A -> Shift 1
Karyawan B -> Shift 3
Karyawan C -> Shift 2


Bin Packing dan Knapsack Problems

In [ ]:
from scipy.optimize import linprog

def bin_packing():
    weights = [7, 5, 6, 3, 8, 4, 2]
    capacity = 10
    bins = []

    while weights:
        bin_weight = 0
        bin_items = []
        for item in sorted(weights, reverse=True):
            if bin_weight + item <= capacity:
                bin_items.append(item)
                bin_weight += item

        for item in bin_items:
            weights.remove(item)

        bins.append(bin_items)

    print(f"Jumlah minimum kotak yang dibutuhkan: {len(bins)}")
    for i, b in enumerate(bins):
        print(f"Kotak {i+1}: {b}")

bin_packing()


Jumlah minimum kotak yang dibutuhkan: 4
Kotak 1: [8, 2]
Kotak 2: [7, 3]
Kotak 3: [6, 4]
Kotak 4: [5]


Graph Optimization

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Membuat data jarak antar kota."""
    data = {}
    data["distance_matrix"] = [
        [0, 10, 15, 20, 25],
        [10, 0, 35, 25, 30],
        [15, 35, 0, 30, 20],
        [20, 25, 30, 0, 15],
        [25, 30, 20, 15, 0]
    ]
    data["num_vehicles"] = 1
    data["depot"] = 0  # Kota awal (A)
    return data

def print_solution(manager, routing, solution):
    """Menampilkan hasil solusi."""
    print("Rute optimal:")
    index = routing.Start(0)
    route = []
    while not routing.IsEnd(index):
        route.append(index)
        index = solution.Value(routing.NextVar(index))
    route.append(routing.End(0))
    print(" -> ".join(map(str, route)))

def solve_tsp():
    """Menyelesaikan masalah Traveling Salesman Problem."""
    data = create_data_model()

    # Membuat Routing Index Manager
    manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]), data["num_vehicles"], data["depot"])

    # Membuat Routing Model
    routing = pywrapcp.RoutingModel(manager)

    # Fungsi jarak antar kota
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Menentukan algoritma pencarian solusi
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    # Menjalankan solver
    solution = routing.SolveWithParameters(search_parameters)

    # Menampilkan hasil
    if solution:
        print_solution(manager, routing, solution)
    else:
        print("Tidak ada solusi yang ditemukan.")

# Menjalankan solver
solve_tsp()


Rute optimal:
0 -> 1 -> 3 -> 4 -> 2 -> 5
